# LangChain Expression Language

In [2]:
import sys
sys.path.insert(1, '../../')
import init_creds as creds
 
AZURE_OPENAI_API_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
# print(AZURE_OPENAI_API_KEY)
# print(AZURE_OPENAI_ENDPOINT)
 
if not AZURE_OPENAI_API_KEY:
    raise ValueError("No AZURE_OPENAI_API_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

In [3]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-mini"
os.environ["AZURE_OPENAI_API_VERSION"]="2024-07-01-preview"


from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

## JSON Formatted Output

In [4]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
response_schemas=[
    ResponseSchema(name="Country",description="Answer to user's query"),
    ResponseSchema(name="ShortName",description="Short name of the country")
]
output_parser=StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser)
format_instructions=output_parser.get_format_instructions()
print(format_instructions)
prompt_template=PromptTemplate(
    template="Give answer to user's query{query}\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions":format_instructions}

)
input_query="Biggest country in the world"

output=model.invoke(prompt_template.format(query=input_query))
print(output)

response_schemas=[ResponseSchema(name='Country', description="Answer to user's query", type='string'), ResponseSchema(name='ShortName', description='Short name of the country', type='string')]
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Country": string  // Answer to user's query
	"ShortName": string  // Short name of the country
}
```
content='```json\n{\n\t"Country": "Russia",\n\t"ShortName": "RU"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 78, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filte

In [5]:
output_parser.parse(output.content)

{'Country': 'Russia', 'ShortName': 'RU'}

## Example LCEL

In [6]:
chain = prompt_template|model|output_parser
chain.invoke(input={"query":input_query})


{'Country': 'Russia', 'ShortName': 'RU'}

In [7]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.vectorstores import FAISS

text = "Noushath worked at Deloitte ,bears like to eat honey"

vectorstore = FAISS.from_texts(
    [text],
    embedding=embedding,
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did Noushath work?")

'Noushath worked at Deloitte.'